In [1]:
import pandas as pd
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob 
from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
# filetuples = glob.glob("/data/shruti/conspirecovery/reddit/initial/tuples/*.gz")
# listtuples = []
# for f in filetuples:
#     temp_F = pd.read_csv(f, header=0, compression='gzip')
#     listtuples.append(temp_F)
    
# tuples = pd.concat(listtuples)
# print(len(tuples))

In [ ]:
print(len(tuples.drop_duplicates(subset=['author','subreddit','created_utc'])))

In [ ]:
tuples = tuples.drop_duplicates(subset=['author','subreddit','created_utc'])

In [ ]:
len(tuples)

In [ ]:
#tuples.to_csv("/data/shruti/conspirecovery/reddit/initial/tuples.csv")

In [ ]:
antivax = ["vaccineskeptics","VaccinesCause","HealthConspiracy", "VaxTalk", "VaccineIngredients"]
climateskeptic = ["climateskeptics","greenskeptics","climatechangebullshit","climatehoax","Globalwarmingisfake","climateconspiratard"]
chemprints = ["CHEMPRINTS","Chemtrail","chemtrails","chemtrailpilots"]
flatearth = ["notaglobe","theworldisflat","flatearth", "Flat_Earth", "Flatearther"]

In [ ]:
def subredditgroup(subreddit):
    if subreddit in antivax:
        return "antivax"
    if subreddit in climateskeptic:
        return "climateskeptic"
    if subreddit in chemprints:
        return "chemtrails"
    if subreddit in flatearth:
        return "flatearth"
    
    return subreddit

In [ ]:
tuples['g_subs'] = tuples['subreddit'].parallel_apply(lambda x: subredditgroup(x))

In [ ]:
data = tuples.copy()
data.sample(5)

In [ ]:
data['rank'] = data.groupby('author')['created_utc'].rank("min", ascending=False)
data.head()

In [ ]:
data.groupby('author')['rank'].apply(max).reset_index().sort_values(by=['rank'], ascending=True)

In [ ]:
maxauthorranks = data.groupby('author')['rank'].apply(max).reset_index().sort_values(by=['rank'], ascending=True)


In [ ]:
print(len(maxauthorranks))
print(len(maxauthorranks.loc[maxauthorranks['rank']>50]))

In [ ]:
valid_authors = maxauthorranks.loc[maxauthorranks['rank']>50]['author'].tolist()

In [ ]:
valid_data = data.loc[data['author'].isin(valid_authors)]

In [ ]:
author2max = maxauthorranks.set_index('author').to_dict()['rank']

valid_data['bin'] = valid_data.apply(lambda x: np.ceil(float(x['rank'])/float(author2max[x['author']]/10.0)), axis=1)
valid_data.head()

In [ ]:
valid_data['bin'].describe()

In [ ]:
#valid_data.to_csv("/data/shruti/conspirecovery/reddit/initial/tuples/binned_tuples.csv")

In [5]:
valid_data = pd.read_csv("/data/shruti/conspirecovery/reddit/initial/tuples/binned_tuples.csv", header=0)

### vaxauthor trajectories

In [6]:
# valid_data = pd.read_csv("../big_data/binned_authorcontribs.csv", header=0)
# valid_data.head()

In [7]:
category = "climateskeptic"

In [8]:
authorVaxcontrib = valid_data.loc[valid_data['g_subs']==category].groupby('author').size().reset_index()
authorVaxcontrib.sample(10)

,author,0
2594,mario2506,9
639,Gorgulak,82
769,JamesBourne008,8
2257,gethereddout,9
284,Captain-Cap,56
2526,lazydonovan,42
275,CalvinCoolest,8
578,Foodei,10
2025,ct_2004,15
1820,battlerythym,6


In [9]:
len(authorVaxcontrib.loc[authorVaxcontrib[0]>5])

3251

In [10]:
vaxAuthors = authorVaxcontrib.loc[authorVaxcontrib[0]>5]['author'].tolist()

In [11]:
vaxData = valid_data.loc[valid_data['author'].isin(vaxAuthors)]
len(vaxData)

16754742

In [12]:
vaxData.sample(5)

,Unnamed: 0,author,created_utc,score,subreddit,type,g_subs,rank,bin
24830690,6395052,Polarisman,1574257110,0.0,EngineeringPorn,comment,EngineeringPorn,59.0,1.0
7169098,7175571,Will_Power,1513790564,2.0,SkyTran,submission,SkyTran,5135.0,2.0
28511752,849500,atlasing,1397042390,3.0,australia,comment,australia,12596.0,10.0
36478928,8838416,PabstyLoudmouth,1354030338,1.0,Browns,comment,Browns,64396.0,8.0
15827719,6607984,mtux96,1438797414,1.0,AskReddit,comment,AskReddit,9376.0,4.0


In [13]:
defaults = pd.read_csv("../lite_data/defaults.csv", header=0)
defaults.head()

,subreddit
0,AskReddit
1,announcements
2,funny
3,pics
4,todayilearned


In [14]:
default_subs = defaults['subreddit'].tolist()

In [15]:
vaxData.loc[~vaxData['g_subs'].isin(default_subs)].groupby('g_subs').size().reset_index().sort_values(by=[0], ascending=False).head(50)

,g_subs,0
28413,The_Donald,513055
16376,Libertarian,322235
36002,climateskeptic,304830
6541,Conservative,301296
36367,conspiracy,206220
35210,canada,134088
46095,nfl,98452
23794,Right_Wing_Politics,91140
5853,Christianity,80864
59235,worldpolitics,79382


In [16]:
vaxData.head()

,Unnamed: 0,author,created_utc,score,subreddit,type,g_subs,rank,bin
0,0,chabanais,1374889166,0.0,Conservative,comment,Conservative,87375.0,8.0
3,3,Patriot4evr,1566312474,6.0,Justfuckmyshitup,comment,Justfuckmyshitup,2518.0,5.0
9,9,Fake_William_Shatner,1533736128,1.0,worldnews,comment,worldnews,20017.0,6.0
10,10,hybrid_remix,1560872065,1.0,heroesofthestorm,comment,heroesofthestorm,991.0,3.0
11,11,kirkisartist,1531524275,0.0,omise_go,comment,omise_go,2953.0,2.0


In [17]:
vaxData.to_csv("../lite_data/climate_tuples.csv")